In [2]:
import json
import os
annotation_path = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019\annotations.json"
with open(annotation_path, "r") as f:
    data = json.load(f)
print("Top-level keys:", list(data.keys()))
if isinstance(data, dict):
    for key in data.keys():
        print(f"\nSample data for '{key}':")
        print(data[key][:2] if isinstance(data[key], list) else data[key])
else:
    print("Sample data:", data[:2])

Top-level keys: ['info', 'licenses', 'categories', 'annotations']

Sample data for 'info':
{'description': 'AUAIR 2019 Dataset', 'url': 'https://bozcani.github.io/AU-AIR-dataset.html', 'version': '1.0', 'year': '2019', 'contributor': 'Ilker Bozcan', 'date_created': '2019/10/01'}

Sample data for 'licenses':
[{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}]

Sample data for 'categories':
['Human', 'Car']

Sample data for 'annotations':
[{'image_name': 'frame_20190829091111_x_0001973.jpg', 'image_width:': 1920.0, 'image_height': 1080.0, 'platform': 'Parrot Bebop 2', 'time': {'year': 2019, 'month': 8, 'day': 29, 'hour': 9, 'min': 11, 'sec': 11, 'ms': 394400.0}, 'longtitude': 10.18798203255313, 'latitude': 56.20630134795274, 'altitude': 19921.6, 'linear_x': 0.03130074199289083, 'linear_y': 0.0283578087

In [4]:
import json
import os

annotation_path = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019\annotations.json"

with open(annotation_path, "r") as f:
    data = json.load(f)

print("Sample annotations (first 2):")
for i, ann in enumerate(data["annotations"][:2]):
    print(f"\nAnnotation {i}:")
    print("Keys:", list(ann.keys()))
    print("Full entry:", ann)

Sample annotations (first 2):

Annotation 0:
Keys: ['image_name', 'image_width:', 'image_height', 'platform', 'time', 'longtitude', 'latitude', 'altitude', 'linear_x', 'linear_y', 'linear_z', 'angle_phi', 'angle_theta', 'angle_psi', 'bbox']
Full entry: {'image_name': 'frame_20190829091111_x_0001973.jpg', 'image_width:': 1920.0, 'image_height': 1080.0, 'platform': 'Parrot Bebop 2', 'time': {'year': 2019, 'month': 8, 'day': 29, 'hour': 9, 'min': 11, 'sec': 11, 'ms': 394400.0}, 'longtitude': 10.18798203255313, 'latitude': 56.20630134795274, 'altitude': 19921.6, 'linear_x': 0.03130074199289083, 'linear_y': 0.028357808757573367, 'linear_z': 0.0744575835764408, 'angle_phi': -0.06713105738162994, 'angle_theta': 0.06894744634628296, 'angle_psi': 1.1161083340644837, 'bbox': [{'top': 163, 'left': 1098, 'height': 185, 'width': 420, 'class': 1}, {'top': 421, 'left': 1128, 'height': 176, 'width': 393, 'class': 1}, {'top': 927, 'left': 1703, 'height': 153, 'width': 183, 'class': 0}]}

Annotation 1:


In [ ]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import numpy as np

#  Paths
root_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019"
annotation_path = os.path.join(root_dir, "annotations.json")
img_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019\images"

#  Category ID mapping (AUAIR to COCO)
label_map = {0: 1, 1: 3, 2: 8, 3: 7, 4: 4, 5: 2, 6: 6, 7: 10}  # AUAIR to COCO
coco_to_auair = {v: k for k, v in label_map.items()}
auair_categories = {
    0: "person",
    1: "car",
    2: "truck",
    3: "train",
    4: "motorcycle",
    5: "bicycle",
    6: "bus",
    7: "traffic light"
}

#  Load and parse annotations
with open(annotation_path, "r") as f:
    data = json.load(f)

# Extract image metadata and flatten annotations
images = []
annotations = []
image_name_to_id = {}
current_image_id = 0
missing_size_count = 0
missing_bbox_count = 0

for ann in data["annotations"]:
    image_name = ann.get("image_name")
    if not image_name:
        print("Warning: Skipping annotation with missing image_name")
        continue
    
    if image_name not in image_name_to_id:
        image_name_to_id[image_name] = current_image_id
        # Use correct key names (image_width:, image_height)
        width = ann.get("image_width:", 1920)
        height = ann.get("image_height", 1080)
        if width == 1920 and height == 1080 and ("image_width:" not in ann or "image_height" not in ann):
            missing_size_count += 1
            print(f"Warning: Missing width/height for {image_name}. Using defaults (1920x1080).")
        
        images.append({
            "id": current_image_id,
            "file_name": image_name,
            "width": width,
            "height": height
        })
        current_image_id += 1
    
    image_id = image_name_to_id[image_name]
    bbox_list = ann.get("bbox", [])
    if not bbox_list:
        missing_bbox_count += 1
        print(f"Warning: No bbox for {image_name}")
        continue
    
    for bbox in bbox_list:
        if not all(key in bbox for key in ["left", "top", "width", "height", "class"]):
            print(f"Warning: Invalid bbox in {image_name}: {bbox}")
            continue
        annotations.append({
            "image_id": image_id,
            "category_id": bbox["class"],
            "bbox": [bbox["left"], bbox["top"], bbox["width"], bbox["height"]],
            "area": bbox["width"] * bbox["height"]
        })

if missing_size_count > 0:
    print(f"Summary: {missing_size_count} images had missing width/height values.")
if missing_bbox_count > 0:
    print(f"Summary: {missing_bbox_count} images had no bounding boxes.")

# Extract categories (from data["categories"] or inferred)
categories = [
    {"id": i, "name": name}
    for i, name in enumerate(data["categories"])
]
# Map to auair_categories
for cat in categories:
    for auair_id, auair_name in auair_categories.items():
        if cat["name"].lower() == auair_name:
            cat["id"] = auair_id
            break

In [6]:
#  Basic Statistics
num_images = len(images)
num_annotations = len(annotations)
num_categories = len(categories)

print(f"Dataset Statistics:")
print(f"  Number of images: {num_images}")
print(f"  Number of annotations: {num_annotations}")
print(f"  Number of categories: {num_categories}")

Dataset Statistics:
  Number of images: 32823
  Number of annotations: 132031
  Number of categories: 8


In [7]:
#  Class Distribution
category_ids = [ann["category_id"] for ann in annotations]
category_counts = Counter(category_ids)
category_df = pd.DataFrame(
    [(auair_categories.get(cid, f"Category {cid}"), count) for cid, count in category_counts.items()],
    columns=["Category", "Count"]
)

plt.figure(figsize=(10, 6))
sns.barplot(x="Count", y="Category", data=category_df.sort_values("Count", ascending=False))
plt.title("Class Distribution")
plt.xlabel("Number of Annotations")
plt.ylabel("Category")
plt.tight_layout()
plt.savefig("class_distribution.png")
plt.close()

In [8]:
#  Image Size Distribution
image_sizes = [(img["width"], img["height"]) for img in images]
widths, heights = zip(*image_sizes)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(widths, bins=20, color="skyblue", edgecolor="black")
plt.title("Image Width Distribution")
plt.xlabel("Width (pixels)")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.hist(heights, bins=20, color="lightgreen", edgecolor="black")
plt.title("Image Height Distribution")
plt.xlabel("Height (pixels)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.savefig("image_size_distribution.png")
plt.close()

In [9]:
#  Bounding Box Size Distribution
bbox_widths = [ann["bbox"][2] for ann in annotations]
bbox_heights = [ann["bbox"][3] for ann in annotations]
bbox_areas = [ann["area"] for ann in annotations]

plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.hist(bbox_widths, bins=20, color="salmon", edgecolor="black")
plt.title("Bounding Box Width Distribution")
plt.xlabel("Width (pixels)")
plt.ylabel("Frequency")

plt.subplot(1, 3, 2)
plt.hist(bbox_heights, bins=20, color="lightcoral", edgecolor="black")
plt.title("Bounding Box Height Distribution")
plt.xlabel("Height (pixels)")
plt.ylabel("Frequency")

plt.subplot(1, 3, 3)
plt.hist(bbox_areas, bins=20, color="plum", edgecolor="black")
plt.title("Bounding Box Area Distribution")
plt.xlabel("Area (pixels²)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.savefig("bbox_size_distribution.png")
plt.close()

In [10]:
#  Objects per Image
objects_per_image = Counter([ann["image_id"] for ann in annotations])
object_counts = list(objects_per_image.values())

plt.figure(figsize=(10, 6))
plt.hist(object_counts, bins=20, color="lightblue", edgecolor="black")
plt.title("Number of Objects per Image")
plt.xlabel("Number of Objects")
plt.ylabel("Frequency")
plt.tight_layout()
plt.savefig("objects_per_image.png")
plt.close()

In [11]:
#  Save EDA Summary
with open("eda_summary.txt", "w") as f:
    f.write("AUAIR2019 Dataset EDA Summary\n")
    f.write("==========================\n\n")
    f.write(f"Number of images: {num_images}\n")
    f.write(f"Number of annotations: {num_annotations}\n")
    f.write(f"Number of categories: {num_categories}\n\n")
    f.write("Class Distribution:\n")
    for _, row in category_df.iterrows():
        f.write(f"  {row['Category']}: {row['Count']}\n")
    f.write("\nImage Size Statistics:\n")
    f.write(f"  Width - Mean: {np.mean(widths):.2f}, Std: {np.std(widths):.2f}\n")
    f.write(f"  Height - Mean: {np.mean(heights):.2f}, Std: {np.std(heights):.2f}\n")
    f.write("\nBounding Box Size Statistics:\n")
    f.write(f"  Width - Mean: {np.mean(bbox_widths):.2f}, Std: {np.std(bbox_widths):.2f}\n")
    f.write(f"  Height - Mean: {np.mean(bbox_heights):.2f}, Std: {np.std(bbox_heights):.2f}\n")
    f.write(f"  Area - Mean: {np.mean(bbox_areas):.2f}, Std: {np.std(bbox_areas):.2f}\n")
    f.write("\nObjects per Image Statistics:\n")
    f.write(f"  Mean: {np.mean(object_counts):.2f}, Std: {np.std(object_counts):.2f}\n")

print("EDA completed. Plots saved as PNG files, and summary saved as 'eda_summary.txt'.")

EDA completed. Plots saved as PNG files, and summary saved as 'eda_summary.txt'.
